In [ ]:
# default_exp FLIR_client_utils
# default_cls_lvl 3

# Flir Camera Client
> Utilities for running a multi camera client
>

In [ ]:
#hide
from nbdev.showdoc import *
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export

import imutils
import cv2
from imutils.video import FPS
import zmq
import numpy as np
import time
import skvideo.io


PORT = 5555
width = 1000
height = 750

def _recv_array(socket:zmq.Context.socket, flags=0, copy=True, track=False):
    """recv a numpy array"""
    md = socket.recv_json(flags=flags)
    msg = socket.recv(flags=flags, copy=copy, track=track)
    buf = memoryview(msg)
    A = np.frombuffer(buf, dtype=md['dtype'])
    # return (A.reshape(md['shape']), md)
    return (A, md)

def _recv_frame(socket, fps):
    """ Receive and process an image from camera"""
    try:
        #  Get the reply.
        topic = socket.recv_string()
        rec_frame, md = _recv_array(socket)
        rec_frame = cv2.imdecode(rec_frame, cv2.IMREAD_GRAYSCALE)
        rec_frame = cv2.cvtColor(rec_frame, cv2.COLOR_BAYER_BG2BGR)
        rec_frame = rec_frame.reshape((3000, 4000, 3))
        # rec_frame = imutils.resize(rec_frame, width=width, height=height)
        # cv2.putText(rec_frame, f'Received frame {md}',
        #             (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        fps.update()

    except Exception as e:
        rec_frame = np.ones((width,height))
        topic = 'cam1'
        # cv2.putText(rec_frame, f'error:  {e}',
        #             (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        print (f"error: message timeout {e}")
        time.sleep(1)
    return topic, rec_frame, md


def _poll_server(url, name):
    context = zmq.Context()
    socket = context.socket(zmq.REQ)
    socket.connect(f"tcp://{url}:{PORT + 1}")
    socket.setsockopt(zmq.LINGER, 0)
    poller = zmq.Poller()
    poller.register(socket, flags=zmq.POLLIN)

    socket.send_string(f"keep_alive {name}")
    result = dict(poller.poll(1000))
    poller.unregister(socket)



def client(name='FrontLeft', url='localhost'):
    """ Received frames from a single camera. Must have the server running"""

    context = zmq.Context()
    
    # subscribe socket
    print( "Connecting to server...")
    socket_sub = context.socket(zmq.SUB)
    socket_sub.connect( f"tcp://{url}:{PORT}")
    socket_sub.setsockopt_string(zmq.SUBSCRIBE, name)
    
    fps = FPS().start()
    
    i = 0
    while True:
        _poll_server(url, name)
        try:
            topic, rec_frame, md = _recv_frame(socket_sub, fps)
            rec_frame = imutils.resize(rec_frame, width=2400, height=1800)

        except KeyboardInterrupt:
            break
        else:
            # cv2.putText(rec_frame, f"{md['framedata']['frameid']}",
            # (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 5)
            cv2.putText(rec_frame, f'Received frame {md}',
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.imshow(topic, rec_frame)
            
            k = cv2.waitKey(10)
            if k == 27 or k == 3:
               break  # esc to quit
                
    fps.stop()
    
    socket_sub.close()
    
    context.term()
    cv2.destroyAllWindows()
    
    print('Finished')
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

def stereo_client(name1='FrontLeft', name2='FrontRight', url='localhost', video='video.avi', vcodec='mjpeg'):
    """ Received frames from two cameras. Must have the server running"""
    context = zmq.Context()
    print( "Connecting to server...")
    socket1 = context.socket(zmq.SUB)
    socket1.connect( f"tcp://{url}:{PORT}"  )
    socket1.setsockopt_string(zmq.SUBSCRIBE, name1)
    socket2 = context.socket(zmq.SUB)
    socket2.connect( f"tcp://{url}:{PORT}"  )
    socket2.setsockopt_string(zmq.SUBSCRIBE, name2)
    
    fps = FPS().start()
    
    i = 0
    
    writer = skvideo.io.FFmpegWriter(video, outputdict={'-vcodec': 'mjpeg'})
    # writer = skvideo.io.FFmpegWriter(outputfile, outputdict={'-vcodec': 'libx264', '-b': '30000000', '-r': "2"})
    
    SHOW_CV_WINDOW = True
    
    stop = False
    while stop == False:
        i += 1
        try:
            _poll_server(url, name1)
            _poll_server(url, name2)
            try:
                topic1, rec_frame1, md1 = _recv_frame(socket1, fps)
                topic2, rec_frame2, md2 = _recv_frame(socket2, fps)
            except Exception as e:
                print(f"Video error:  {e}")
            else:
                cv2.putText(rec_frame1, f"{md1['framedata']['frameid']}",
                            (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 5)
                cv2.putText(rec_frame2, f"{md2['framedata']['frameid']}",
                            (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 5)
    
                s_frame = np.concatenate((rec_frame1, rec_frame2), axis=1)
                writer.writeFrame(s_frame)
                print(f"Writing Frame pair {md1['framedata']['frameid']}, {md2['framedata']['frameid']}")
    
                if SHOW_CV_WINDOW:
                    rec_frame1 = imutils.resize(rec_frame1, width=width, height=height)
                    rec_frame2 = imutils.resize(rec_frame2, width=width, height=height)
                    # cv2.putText(rec_frame1, f"{md2['framedata']['frameid']}",
                    #             (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                    # cv2.putText(rec_frame2, f"{md2['framedata']['frameid']}",
                    #             (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
                    cv2.imshow(topic1, rec_frame1)
                    cv2.imshow(topic2, rec_frame2)
                    k = cv2.waitKey(10)
                    if k == 27 or k == 3:
                       stop = True
        except KeyboardInterrupt:
            stop = True
            print("Stopping")
    
    fps.stop()
    writer.close()
    socket1.close()
    socket2.close()
    context.term()
    cv2.destroyAllWindows()
    
    print('Finished')
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()/2))

        

## Examples
### Example of a single camera client
__flir-client.py__

In [ ]:
from  boxfish_flir_cam import FLIR_client_utils
if __name__== "__main__":

    FLIR_client_utils.client(name='FrontLeft', url='localhost')
    

Connecting to server...
Finished
[INFO] approx. FPS: 1.67


### Example of a Stereo camera client
__flir-stereo-client.py__

In [ ]:
from  boxfish_flir_cam import FLIR_client_utils
if __name__== "__main__":

    FLIR_client_utils.stereo_client(name1='FrontLeft', name2='FrontRight', url='localhost', video='video.avi', vcodec='mjpeg')






Connecting to server...
Writing Frame pair 29, 29
Writing Frame pair 30, 30
Writing Frame pair 31, 31
Writing Frame pair 32, 32
Writing Frame pair 33, 33
Writing Frame pair 34, 34
Writing Frame pair 35, 35
Writing Frame pair 36, 36
Writing Frame pair 37, 37
Writing Frame pair 38, 38
Writing Frame pair 39, 39
Writing Frame pair 40, 40
Finished
[INFO] approx. FPS: 1.72
